# Aerosol-ice nucleating particle closure

Contributed by Yijia Sun and Daniel Knopf

---

## Overview
By inputting the particle size distribution (PSD) information measured from COMBLE AND ZEPPELIN cases in our quickcalc module and applying with different immersion freezing (IMF) parameterizations, we can derive the number concentration of predicted ice nucleating particles (INPs). The comparison between the predicted INPs and observed INPs can be referred to as a simplistic closure study. Here, we chose the PSD of COMBLE case in 18 UTC March 13 and ZEPPELIN case in 0 UTC March 13 and performed the simulation with the assumption that all aerosols are sea spray aerosols (SSA), number concentration of aerosols in accumaltion mode is halved and aerosols in aitken mode are neglected. Three different IMF parameterizations were tested, MC2018 (singular ice nucleation active sites-based parameterization of SSA from Fig. 8 caption in McCluskey et al., JGR, 2018), PT2022 (fitted singular ice nucleation active sites-based parameterization of SSA in Peter et al., Science Advances, 2022) and ABIFM (CNT-based parameterization using water activity, parameters are from Fig. 5 in Peter et al., Science Advances, 2022). Figures of results are attached, <a href="https://photos.app.goo.gl/VyqB9fbr8vxtifhG9"> here</a>.


## Prerequisites
**System requirements**:
Be sure to implement numpy, xarray, pandas, pint and flake8 in your python environment and run this jupyter notebook in <a href="https://drive.google.com/drive/folders/1inyzNYSXqU_QP32m_fm4YJbPPOsw6QnA?usp=sharing"> this folder</a>. And we used matlab to get our finalized figures.

---

## Imports

In [ ]:
from quickcalc import quickcalc
import numpy as np
from time import time
import xarray as xr
import pandas as pd

PSD parameters are all stored in a google sheet <a href="https://docs.google.com/spreadsheets/d/15PzDQ6rPBRDpDK9ZABW7mCNFqrILRrFvMRdxVN36VJA/edit?usp=sharing">here</a>PSD parameters are all stored in a google sheet <a href="https://docs.google.com/spreadsheets/d/15PzDQ6rPBRDpDK9ZABW7mCNFqrILRrFvMRdxVN36VJA/edit?usp=sharing">here</a>

In [ ]:
aer_info1= {"name": "COMBLE18UTCMarch13",             #COMBLE18UTCMarch13case
                     "n_init_max": [21.66e6, 7.14e6],
                      "psd": {"type": "multi_logn",
                              "diam_mean": [0.13e-6,0.33e-6],
                              "geom_sd":[1.67,1.69],
                              "n_bins":50,
                              "diam_min":(0.0078e-6,5.577e-6),
                              "m_ratio":1.5},
                              "nucleus_type": "SSA"}

aer_info2= {"name": "ZEPPELIN0UTCMarch13",             #ZEPPELIN0UTCMarch13case
                     "n_init_max": [100.605e6, 24.58e6],
                      "psd": {"type": "multi_logn",
                              "diam_mean": [0.13e-6,0.61e-6],
                              "geom_sd":[2.2,1.9],
                              "n_bins":50,
                              "diam_min":(0.0059e-6,11.59e-6),
                              "m_ratio":1.5},
                              "nucleus_type": "SSA"}

## Calculate the predicted INPs
The activation time of 1min for ABIFM is the same as mentioned in Peter et al., Science Advances, 2022. And all implemented equations for IMF parameterizations can be found in AER.py.

In [ ]:
qqq1=[]
qqq2=[]
qqq3=[]
ttt1=[]
ttt2=[]
ttt3=[]


for T in range(1, 48, 1):
        #COMBLE18UTCMarch13case
        qq1 = quickcalc(aer_info_dict={**aer_info1, **{"singular_fun": "PT2022"}}, T_in=243+(T-1)*0.5, use_ABIFM=False,RH_in=99.5)  
        qqq1.append(qq1.variables['inp_tot'].values)
        qq2 = quickcalc(aer_info_dict={**aer_info1, **{"singular_fun": "MC2018"}}, T_in=243+(T-1)*0.5, use_ABIFM=False,RH_in=99.5)  
        qqq2.append(qq2.variables['inp_tot'].values)
        qq3 =quickcalc(aer_info_dict=aer_info1, T_in=243+(T-1)*0.5, use_ABIFM=True,RH_in=99.5, ABIFM_delta_t=60.)  
        qqq3.append(qq3.variables['inp_tot'].values)
        #ZEPPELIN0UTCMarch13case
        tt1 = quickcalc(aer_info_dict={**aer_info2, **{"singular_fun": "PT2022"}}, T_in=243+(T-1)*0.5, use_ABIFM=False,RH_in=99.5)  
        ttt1.append(tt1.variables['inp_tot'].values)
        tt2 = quickcalc(aer_info_dict={**aer_info2, **{"singular_fun": "MC2018"}}, T_in=243+(T-1)*0.5, use_ABIFM=False,RH_in=99.5)  
        ttt2.append(tt2.variables['inp_tot'].values)
        tt3 =quickcalc(aer_info_dict=aer_info2, T_in=243+(T-1)*0.5, use_ABIFM=True,RH_in=99.5, ABIFM_delta_t=60.)  
        ttt3.append(tt3.variables['inp_tot'].values)
        

## Check all varaibles in the output netcdf file

qq1

## Export the dataset of predicted INPs

In [ ]:
test=pd.DataFrame(data=qqq1)
test.to_csv('SSAPT2022COMBLE18UTCMarch13.csv')
test=pd.DataFrame(data=qqq2)
test.to_csv('SSAMC2018COMBLE18UTCMarch13.csv')
test=pd.DataFrame(data=qqq3)
test.to_csv('SSAABIFMCOMBLE18UTCMarch13.csv')
test=pd.DataFrame(data=ttt1)
test.to_csv('SSAPT2022ZEPPELIN0UTCMarch13.csv')
test=pd.DataFrame(data=ttt2)
test.to_csv('SSAMC2018ZEPPELIN0UTCMarch13.csv')
test=pd.DataFrame(data=ttt3)
test.to_csv('SSAABIFMZEPPELIN0UTCMarch13.csv')

## Compare the predicted INPs with observed INPs
The number concentration of observed INPs can be found in Sample details: 3.12.20 - 3.14.20 in 17 CAOS from <a href="https://docs.google.com/spreadsheets/d/1eYupJDFb1VFpXFJvR-ZZo-onJJvEgGa7JTm7GsVOrCk/edit?usp=sharing">here</a>. By running the <a href="https://drive.google.com/drive/folders/1yO6psVcruXTB0DRIItP5DzzB3jXYFU2a?usp=sharing">matlab code</a> named with "COMBLE" and "ZEPPELIN", you can easily get the figures of the results.

---

## Summary
Based on our assumption (all aerosols are sea spray aerosols (SSA), number concentration of aerosols in accumaltion mode is halved and aerosols in aitken mode are neglected), in COMBLE18UTCMarch13case, the predicted INPs applied with ABIFM and PT2022 are mostly close to observed INPs. While in ZEPPELIN0UTCMarch13, the predicted INPs applied with MC2018 are mostly close to observed INPs.

## References
- Thanks for Israel who wrote the original version of quickcalc module, we can easily derive the number concentration of predicted ice nucleating particles with different IMF parameterizations and we thank Ann for supplying the dataset of observed INPs and aerosol PSD parameters. 

- Knopf et al. (A 1D model for nucleation of ice from aerosol particles: an application to mixed-phase arctic stratus, submitted to Journal of Advances in Modeling Earth Systems, 2023) used a simplified 1D aerosol-cloud model informed by a large-eddy simulation (LES) to examine the ice nucleating particle (INP) reservoir dynamics of Arctic mixed-phase clouds (MPCs) during the Surface Heat Budget of the Arctic campaign (SHEBA), already showed the potential of this 1D model in deriving the predicted INP with different IMF parameterizations.

- A manuscript is currently in preparation (Sun et al., *in prep*). This manuscript further investigates the impacts of different types of aerosol properties, IMF parameterizations, and various model sensitivity parameters on the MPCs processes.

- Contact yijia.sun@stonybrook.edu if you have any questions.